# CHATBOT WITH DIALOGPT

In [ ]:
!pip install transformers datasets tensorflow

## GPU info

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 19 14:08:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   57C    P0              29W /  72W |    191MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Check if GPU is available
from tensorflow.python.client import device_lib

def get_gpu_details():
    devices = device_lib.list_local_devices()
    for device in devices:
        if device.device_type == 'GPU':
            print(f"Device Name: {device.name}")
            print(f"Memory Limit: {device.memory_limit} bytes")
            print(f"Description: {device.physical_device_desc}")

get_gpu_details()


Device Name: /device:GPU:0
Memory Limit: 21991653376 bytes
Description: device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


### Import libraries

In [10]:
import os
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForCausalLM, DataCollatorForLanguageModeling, create_optimizer
import tensorflow as tf
from datasets import Dataset

### Load the Dataset

In [11]:
# Loading the DataFrame
data_dir = os.path.join(os.getcwd(), 'data_dd')
file_path_parquet = os.path.join(data_dir, 'training_df_dd.parquet')
training_data_final = pd.read_parquet(file_path_parquet)

training_data_final.head(10)

,input,response,encoder_input_data,decoder_input_data,decoder_output_data
0,say how about going for a few beers after dinner,you know that is tempting but is really not go...,"[0, 0, 0, 0, 0, 138, 33, 37, 75, 20, 8, 206, 3...","[0, 0, 1, 5, 46, 11, 9, 3717, 29, 9, 60, 15, 4...","[0, 0, 5, 46, 11, 9, 3717, 29, 9, 60, 15, 47, ..."
1,you know that is tempting but is really not go...,what do you mean it will help us to relax,"[0, 0, 5, 46, 11, 9, 3717, 29, 9, 60, 15, 47, ...","[0, 0, 0, 0, 0, 1, 18, 13, 5, 161, 10, 23, 101...","[0, 0, 0, 0, 0, 18, 13, 5, 161, 10, 23, 101, 9..."
2,what do you mean it will help us to relax,do you really think so i do not it will just m...,"[0, 0, 0, 0, 0, 18, 13, 5, 161, 10, 23, 101, 9...","[1, 13, 5, 60, 43, 36, 4, 13, 15, 10, 23, 48, ...","[13, 5, 60, 43, 36, 4, 13, 15, 10, 23, 48, 102..."
3,do you really think so i do not it will just m...,i guess you are right but what shall we do i d...,"[13, 5, 60, 43, 36, 4, 13, 15, 10, 23, 48, 102...","[1, 4, 226, 5, 17, 53, 29, 18, 325, 22, 13, 4,...","[4, 226, 5, 17, 53, 29, 18, 325, 22, 13, 4, 13..."
4,i guess you are right but what shall we do i d...,i suggest a walk over to the gym where we can ...,"[4, 226, 5, 17, 53, 29, 18, 325, 22, 13, 4, 13...","[1, 4, 593, 8, 423, 140, 7, 6, 973, 105, 22, 2...","[4, 593, 8, 423, 140, 7, 6, 973, 105, 22, 21, ..."
5,i suggest a walk over to the gym where we can ...,that 's a good idea i hear mary and sally ofte...,"[4, 593, 8, 423, 140, 7, 6, 973, 105, 22, 21, ...","[1, 11, 38, 8, 47, 179, 4, 237, 441, 14, 3323,...","[11, 38, 8, 47, 179, 4, 237, 441, 14, 3323, 30..."
6,that 's a good idea i hear mary and sally ofte...,sounds great to me if they are willing we coul...,"[11, 38, 8, 47, 179, 4, 237, 441, 14, 3323, 30...","[1, 154, 99, 7, 26, 57, 54, 17, 1083, 22, 79, ...","[154, 99, 7, 26, 57, 54, 17, 1083, 22, 79, 200..."
7,sounds great to me if they are willing we coul...,good let us go now,"[154, 99, 7, 26, 57, 54, 17, 1083, 22, 79, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 47, 74, 93, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 47, 74, 93, 59,..."
8,good let us go now,all right,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 47, 74, 93, 59,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 50,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 53..."
9,can you do push ups,of course i can it is a piece of cake believe ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 5, 13, 1635...","[1, 16, 125, 4, 21, 10, 9, 8, 773, 16, 899, 25...","[16, 125, 4, 21, 10, 9, 8, 773, 16, 899, 254, ..."


### Tokenize and Prepare Data for Training

In [20]:
# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

# Initialize the tokenizer and model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForCausalLM.from_pretrained(model_name)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize input and response
def tokenize_function(example):
    input_ids = tokenizer.encode(example['input'] + tokenizer.eos_token, truncation=True, padding='max_length', max_length=17)
    response_ids = tokenizer.encode(example['response'] + tokenizer.eos_token, truncation=True, padding='max_length', max_length=17)
    return {'input_ids': input_ids, 'response_ids': response_ids}

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(training_data_final)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Prepare the data for training
def format_dataset(data):
    input_ids = data['input_ids']
    response_ids = data['response_ids']
    labels = response_ids.copy()

    # Concatenate input and response
    input_ids.extend(response_ids)
    return {'input_ids': input_ids, 'labels': labels}

formatted_dataset = tokenized_dataset.map(format_dataset, remove_columns=dataset.column_names)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Map:   0%|          | 0/89861 [00:00<?, ? examples/s]

Map:   0%|          | 0/89861 [00:00<?, ? examples/s]

### Train the Model

In [21]:
# Convert the dataset to a TensorFlow dataset
tf_dataset = formatted_dataset.to_tf_dataset(
    columns=['input_ids', 'labels'],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator,
)

# Define optimizer
num_train_steps = len(tf_dataset) * 3  # Assuming 3 epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=0
)

# Compile the model
model.compile(optimizer=optimizer)

# Train the model
model.fit(tf_dataset, epochs=3)

Epoch 1/3
44931/44931 [==============================] - 5881s 129ms/step - loss: 3.0427
Epoch 2/3
44931/44931 [==============================] - 5799s 129ms/step - loss: 2.0909
Epoch 3/3
44931/44931 [==============================] - 5776s 129ms/step - loss: 1.5407


### Save the Trained Model

In [26]:
# Save the model
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

### Generate Responses

In [ ]:
# Load the trained model and tokenizer
model = TFAutoModelForCausalLM.from_pretrained('./trained_model')
tokenizer = AutoTokenizer.from_pretrained('./trained_model')
tokenizer.padding_side = "left"  # Set padding side to left

# Generate a response
def generate_response(input_text):
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='tf', truncation=True, padding='max_length', max_length=256)
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

test_examples = [
    "How are you doing today?",
    "What is your name?",
    "Can you help me with my homework?",
    "What is the weather like?",
    "Tell me a joke.",
    "Who is the president of the United States?",
    "What is the capital of France?",
    "Do you like pizza?",
    "What is your favorite color?",
    "Goodbye!"
]

for text in test_examples:
    print(f"Input: {text}")
    print(f"Response: {generate_response(text)}\n")


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./trained_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Input: How are you doing today?
